# de base de datos a json

In [288]:

from fastapi import FastAPI, requests, HTTPException
import uvicorn
import os
from dotenv import load_dotenv
import psycopg2
import json
import pandas as pd
load_dotenv()
    
#Leer Las Variables:
host = os.getenv("DB_HOST_AWS")
username = os.getenv("DB_USER_AWS")
password = os.getenv("DB_PASSWORD_AWS")
database = os.getenv("DB_DATABASE_AWS")
port = os.getenv("DB_PORT_AWS")
try:
    conn = psycopg2.connect(host=host,
                                database=database,
                                user=username,
                                password=password,
                                port=port,
                                sslmode="require")
    
    print("Conexión exitosa a la base de datos PostgreSQL con SSL")
except psycopg2.OperationalError as e:
    raise HTTPException(status_code=400, detail="Error de conexión: " + str(e))
except Exception as error:
    raise HTTPException(status_code=400, detail="Error desconocido: " + str(e))
cur = conn.cursor()


Conexión exitosa a la base de datos PostgreSQL con SSL


In [289]:
# Escribe la consulta SQL
query = """SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public' AND table_type = 'BASE TABLE';
"""
# query = "SELECT * FROM preguntas_front"

# Usa pandas para ejecutar la consulta y convertirla en un DataFrame
df = pd.read_sql_query(query, conn)
df.head(50)

C:\Users\2012f\AppData\Local\Temp\ipykernel_8440\4123380349.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,table_name
0,no_sociosanit_formulario
1,opciones_chatbot
2,admin_data
3,preguntas_opciones_chatbot
4,categorias_chatbot
5,preguntas
6,sociosanitarios_formulario
7,preguntas_chatbot
8,preguntas_front
9,categoria_pregunta_chat_intermed


In [277]:
# Escribe la consulta SQL
query = """
        SELECT * FROM categorias_chatbot
        """
df = pd.read_sql_query(query, conn)
df.head(50)

C:\Users\2012f\AppData\Local\Temp\ipykernel_8440\3971965006.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,id_categoria,titulo_categoria
0,1,Personal Sanitario
1,2,Trabajador Social
2,3,Psicologo
3,4,Educador
4,5,Voluntarios y Cuidadores
5,6,Tengo vih
6,7,Creo que me he expuesto al virus
7,8,Quiero saber mas sobre el vih/sida
8,9,Estoy apoyando a una persona seropositiva


# Cargar datos

In [264]:
# Recuperar los id_pregunta de la tabla preguntas_chatbot
cur.execute("SELECT id_pregunta, texto_pregunta FROM preguntas_chatbot")
preguntas_dict = cur.fetchall()  # Esto es una lista de tuplas con (id_pregunta, pregunta)

# Recuperar los id_opcion de la tabla opciones_chatbot
cur.execute("SELECT id_opcion, texto_opcion FROM opciones_chatbot")
opciones_dict = cur.fetchall()  # Esto es una lista de tuplas con (id_opcion, opcion)

# Crear mapeo de preguntas a ids
preguntas_dict = {pregunta[1]: pregunta[0] for pregunta in preguntas_dict}

# Crear mapeo de opciones a ids
opciones_dict = {opcion[1]: opcion[0] for opcion in opciones_dict}

print(preguntas_dict)
print(opciones_dict)



{'¿Qué apoyo necesitas?': 1, '¿Qué necesitas como Trabajador Social?': 2, '¿Cuándo ocurrió la posible infección?': 3, '¿Necesitas recursos de referencia?': 4, '¿Tienes acceso a un médico?': 5, '¿Has compartido tu preocupación con alguien?': 6, '¿Qué necesitas como Psicologo?': 7, '¿Ha sido en un entorno de "chem-sex"?': 8, '¿Estás en tratamiento TAR?': 9, '¿Has compartido tu diagnostico con alguien?': 10, '¿Tiene acceso a recursos locales o grupos de apoyo?': 11, '¿Qué necesitas como Voluntario/Cuidador?': 12, '¿Cuándo te diagnosticaron?': 13, '¿Has compartido su preocupación sobre esta persona con alguien?': 14, '¿Sabes qué es la PEP?': 15, '¿Qué necesitas como Personal Sanitario?': 16, '¿Qué tipo de exposición fue?': 17, '¿Qué necesitas como Educador?': 18, '¿Quieres información sobre algun tema?': 19}
{'Manejo clínico de pacientes con vih': 1, 'Protocolo PEP': 2, 'Tratamientos (PREP, TAR)': 3, 'Prevención de infecciones oportunistas': 4, 'Consejería para adherencia al tratamiento': 

In [265]:
# Lista de relaciones entre preguntas y opciones (basado en los valores que tienes)
# Lista de relaciones entre preguntas y opciones con los IDs correctos
preguntas_opciones = [
    (preguntas_dict['¿Qué necesitas como Personal Sanitario?'], opciones_dict['Manejo clínico de pacientes con vih']),
    (preguntas_dict['¿Qué necesitas como Personal Sanitario?'], opciones_dict['Protocolo PEP']),
    (preguntas_dict['¿Qué necesitas como Personal Sanitario?'], opciones_dict['Tratamientos (PREP, TAR)']),
    (preguntas_dict['¿Qué necesitas como Personal Sanitario?'], opciones_dict['Prevención de infecciones oportunistas']),
    (preguntas_dict['¿Qué necesitas como Personal Sanitario?'], opciones_dict['Consejería para adherencia al tratamiento']),

    (preguntas_dict['¿Qué necesitas como Trabajador Social?'], opciones_dict['Acceso a medicamentos o servicios']),
    (preguntas_dict['¿Qué necesitas como Trabajador Social?'], opciones_dict['Recursos legales y derechos']),
    (preguntas_dict['¿Qué necesitas como Trabajador Social?'], opciones_dict['Apoyo a personas en situación de vulnerabilidad']),
    (preguntas_dict['¿Qué necesitas como Trabajador Social?'], opciones_dict['Conexión con grupos de apoyo comunitario']),
    (preguntas_dict['¿Qué necesitas como Trabajador Social?'], opciones_dict['Información sobre redes de Servicios Sociales']),

    (preguntas_dict['¿Qué necesitas como Psicologo?'], opciones_dict['Apoyo emocional para personas recién diagnosticadas']),
    (preguntas_dict['¿Qué necesitas como Psicologo?'], opciones_dict['Intervenciones para adherencia al tratamiento']),
    (preguntas_dict['¿Qué necesitas como Psicologo?'], opciones_dict['Manejo del estigma y problemas de salud mental']),
    (preguntas_dict['¿Qué necesitas como Psicologo?'], opciones_dict['Recursos para pacientes con vih y trastornos psicológicos']),
    (preguntas_dict['¿Qué necesitas como Psicologo?'], opciones_dict['Consejería en prevención y autocuidado']),

    (preguntas_dict['¿Qué necesitas como Educador?'], opciones_dict['Material educativo sobre vih']),
    (preguntas_dict['¿Qué necesitas como Educador?'], opciones_dict['Capacitación en prevención']),
    (preguntas_dict['¿Qué necesitas como Educador?'], opciones_dict['Métodos para combatir el estigma']),
    (preguntas_dict['¿Qué necesitas como Educador?'], opciones_dict['Recursos para sensibilización']),
    (preguntas_dict['¿Qué necesitas como Educador?'], opciones_dict['Estadísticas y datos actualizados']),

    (preguntas_dict['¿Qué necesitas como Voluntario/Cuidador?'], opciones_dict['Info básica sobre vih']),
    (preguntas_dict['¿Qué necesitas como Voluntario/Cuidador?'], opciones_dict['Consejos para apoyar emocionalmente']),
    (preguntas_dict['¿Qué necesitas como Voluntario/Cuidador?'], opciones_dict['Recursos legales y sociales para pacientes']),
    (preguntas_dict['¿Qué necesitas como Voluntario/Cuidador?'], opciones_dict['Conexión con grupos de apoyo comunitario']),
    (preguntas_dict['¿Qué necesitas como Voluntario/Cuidador?'], opciones_dict['Métodos de autocuidado para cuidadores']),
    
    (preguntas_dict['¿Cuándo te diagnosticaron?'], opciones_dict['Hace menos de 6 meses']),
    (preguntas_dict['¿Cuándo te diagnosticaron?'], opciones_dict['Entre 6 meses y un año']),
    (preguntas_dict['¿Cuándo te diagnosticaron?'], opciones_dict['Hace más de un año']),
    
    (preguntas_dict['¿Estás en tratamiento TAR?'], opciones_dict['Sí']),
    (preguntas_dict['¿Estás en tratamiento TAR?'], opciones_dict['No']),
    (preguntas_dict['¿Estás en tratamiento TAR?'], opciones_dict['No estoy seguro']),

    (preguntas_dict['¿Tienes acceso a un médico?'], opciones_dict['Sí']),
    (preguntas_dict['¿Tienes acceso a un médico?'], opciones_dict['No']),

    (preguntas_dict['¿Has compartido tu diagnostico con alguien?'], opciones_dict['Un amigo']),
    (preguntas_dict['¿Has compartido tu diagnostico con alguien?'], opciones_dict['Algún familiar']),
    (preguntas_dict['¿Has compartido tu diagnostico con alguien?'], opciones_dict['Mi pareja en ese momento']),
    (preguntas_dict['¿Has compartido tu diagnostico con alguien?'], opciones_dict['Compañero/a de trabajo']),
    (preguntas_dict['¿Has compartido tu diagnostico con alguien?'], opciones_dict['Con mi jefe/a']),
    (preguntas_dict['¿Has compartido tu diagnostico con alguien?'], opciones_dict['Personal de ONG']),
    (preguntas_dict['¿Has compartido tu diagnostico con alguien?'], opciones_dict['Expareja']),
    (preguntas_dict['¿Has compartido tu diagnostico con alguien?'], opciones_dict['Nadie']),

    (preguntas_dict['¿Quieres información sobre algun tema?'], opciones_dict['Opciones de tratamiento']),
    (preguntas_dict['¿Quieres información sobre algun tema?'], opciones_dict['Apoyo psicológico']),
    (preguntas_dict['¿Quieres información sobre algun tema?'], opciones_dict['Derechos laborales y legales']),
    (preguntas_dict['¿Quieres información sobre algun tema?'], opciones_dict['Grupos de apoyo']),
    (preguntas_dict['¿Quieres información sobre algun tema?'], opciones_dict['Prevención de transmisión']),

    (preguntas_dict['¿Cuándo ocurrió la posible infección?'], opciones_dict['Últimas 72h']),
    (preguntas_dict['¿Cuándo ocurrió la posible infección?'], opciones_dict['Hace más de 72h']),

    (preguntas_dict['¿Qué tipo de exposición fue?'], opciones_dict['Relación sexual']),
    (preguntas_dict['¿Qué tipo de exposición fue?'], opciones_dict['Aguja compartida']),
    (preguntas_dict['¿Qué tipo de exposición fue?'], opciones_dict['Contacto con fluidos corporales (sangre, fluido sexual..)']),
    (preguntas_dict['¿Qué tipo de exposición fue?'], opciones_dict['No estoy seguro']),

    (preguntas_dict['¿Ha sido en un entorno de "chem-sex"?'], opciones_dict['Sí']),
    (preguntas_dict['¿Ha sido en un entorno de "chem-sex"?'], opciones_dict['No']),

    (preguntas_dict['¿Tienes acceso a un médico?'], opciones_dict['Sí']),
    (preguntas_dict['¿Tienes acceso a un médico?'], opciones_dict['No']),

    (preguntas_dict['¿Has compartido tu preocupación con alguien?'], opciones_dict['La persona que me preocupa']),
    (preguntas_dict['¿Has compartido tu preocupación con alguien?'], opciones_dict['Un amigo']),
    (preguntas_dict['¿Has compartido tu preocupación con alguien?'], opciones_dict['Algún familiar']),
    (preguntas_dict['¿Has compartido tu preocupación con alguien?'], opciones_dict['Mi pareja en ese momento']),
    (preguntas_dict['¿Has compartido tu preocupación con alguien?'], opciones_dict['Compañero/a de trabajo']),
    (preguntas_dict['¿Has compartido tu preocupación con alguien?'], opciones_dict['Con mi jefe/a']),
    (preguntas_dict['¿Has compartido tu preocupación con alguien?'], opciones_dict['Personal de ONG']),
    (preguntas_dict['¿Has compartido tu preocupación con alguien?'], opciones_dict['Expareja']),
    (preguntas_dict['¿Has compartido tu preocupación con alguien?'], opciones_dict['Nadie']),

    (preguntas_dict['¿Sabes qué es la PEP?'], opciones_dict['Si, quiero más información']),
    (preguntas_dict['¿Sabes qué es la PEP?'], opciones_dict['No ¿Qué es?']),

    (preguntas_dict['¿Necesitas recursos de referencia?'], opciones_dict['¿Qué es el vih/sida?']),
    (preguntas_dict['¿Necesitas recursos de referencia?'], opciones_dict['Formas de transmisión']),
    (preguntas_dict['¿Necesitas recursos de referencia?'], opciones_dict['Métodos de prevención']),
    (preguntas_dict['¿Necesitas recursos de referencia?'], opciones_dict['Impacto del tratamiento']),
    (preguntas_dict['¿Necesitas recursos de referencia?'], opciones_dict['Historia del vih']),

    (preguntas_dict['¿Tiene acceso a recursos locales o grupos de apoyo?'], opciones_dict['Sí']),
    (preguntas_dict['¿Tiene acceso a recursos locales o grupos de apoyo?'], opciones_dict['No']),

    (preguntas_dict['¿Has compartido su preocupación sobre esta persona con alguien?'], opciones_dict['La persona que me preocupa']),
    (preguntas_dict['¿Has compartido su preocupación sobre esta persona con alguien?'], opciones_dict['Un amigo']),
    (preguntas_dict['¿Has compartido su preocupación sobre esta persona con alguien?'], opciones_dict['Algún familiar']),
    (preguntas_dict['¿Has compartido su preocupación sobre esta persona con alguien?'], opciones_dict['Mi pareja en ese momento']),
    (preguntas_dict['¿Has compartido su preocupación sobre esta persona con alguien?'], opciones_dict['Compañero/a de trabajo']),
    (preguntas_dict['¿Has compartido su preocupación sobre esta persona con alguien?'], opciones_dict['Con mi jefe/a']),
    (preguntas_dict['¿Has compartido su preocupación sobre esta persona con alguien?'], opciones_dict['Personal de ONG']),
    (preguntas_dict['¿Has compartido su preocupación sobre esta persona con alguien?'], opciones_dict['Expareja']),
    (preguntas_dict['¿Has compartido su preocupación sobre esta persona con alguien?'], opciones_dict['Nadie']),

]


for pregunta_id, opcion_id in preguntas_opciones:
    cur.execute("""
        SELECT 1 FROM preguntas_opciones_chatbot
        WHERE id_pregunta = %s AND id_opcion = %s;
    """, (pregunta_id, opcion_id))
    
    # Si no existe la combinación, insertar
    if not cur.fetchone():
        cur.execute("""
            INSERT INTO preguntas_opciones_chatbot (id_pregunta, id_opcion)
            VALUES (%s, %s);
        """, (pregunta_id, opcion_id))

# Confirmar los cambios en la base de datos
conn.commit()


In [280]:
cur = conn.cursor()

# Query para eliminar la tabla
query = """
ALTER TABLE categorias_chatbot
ADD COLUMN seccion VARCHAR(255);
"""

# Ejecutar la eliminación
cur.execute(query)
conn.commit()

# JUNTAR EN UN TABLA CATEGORIAS Y OPCIONES

## Usuarios

In [ ]:
# Escribe la consulta SQL
query_usuarios = """
            SELECT 
                c.id_categoria,
                p.id_pregunta,
                o.id_opcion,
                c.titulo_categoria,
                p.texto_pregunta,
                o.texto_opcion
            FROM 
                categorias_chatbot c
            JOIN 
                categoria_pregunta_chat_intermed cp ON c.id_categoria = cp.id_categoria
            JOIN 
                preguntas_chatbot p ON cp.id_pregunta = p.id_pregunta
            JOIN 
                preguntas_opciones_chatbot po ON p.id_pregunta = po.id_pregunta
            JOIN 
                opciones_chatbot o ON po.id_opcion = o.id_opcion
            WHERE 
                c.seccion = 'usuario'
            ORDER BY 
                c.id_categoria, p.id_pregunta, o.id_opcion;
            """


df = pd.read_sql_query(query_usuarios, conn)

# Convertir el DataFrame a JSON
# Opción 1: Convertir a un diccionario JSON estructurado por filas
json_data = df.to_json(orient='records', indent=4)

# Mostrar el JSON por consola (opcional)
print(json_data)

IndentationError: unexpected indent (1079041054.py, line 37)

In [295]:
# Escribe la consulta SQL
query_sociosanitarios = """
    SELECT 
        c.id_categoria,
        p.id_pregunta,
        o.id_opcion,
        c.titulo_categoria,
        p.texto_pregunta,
        o.texto_opcion
    FROM 
        categorias_chatbot c
    JOIN 
        categoria_pregunta_chat_intermed cp ON c.id_categoria = cp.id_categoria
    JOIN 
        preguntas_chatbot p ON cp.id_pregunta = p.id_pregunta
    JOIN 
        preguntas_opciones_chatbot po ON p.id_pregunta = po.id_pregunta
    JOIN 
        opciones_chatbot o ON po.id_opcion = o.id_opcion
    WHERE 
        c.seccion = 'sociosanitario'
    ORDER BY 
        c.id_categoria, p.id_pregunta, o.id_opcion;
"""

# Ejecutar la consulta y obtener el DataFrame
df = pd.read_sql_query(query_sociosanitarios, conn)

# Agrupar las opciones dentro de las preguntas
grouped_questions = (
    df.groupby(['id_categoria', 'titulo_categoria', 'id_pregunta', 'texto_pregunta'])
    .apply(lambda x: x[['id_opcion', 'texto_opcion']].to_dict(orient='records'))
    .reset_index()
    .rename(columns={0: 'opciones'})
)

# Convertir las preguntas a una lista estructurada por categorías
result = (
    grouped_questions.groupby(['id_categoria', 'titulo_categoria'])
    .apply(lambda x: x[['id_pregunta', 'texto_pregunta', 'opciones']].to_dict(orient='records'))
    .reset_index()
    .rename(columns={0: 'preguntas'})
    .to_dict(orient='records')
)

# Convertir el resultado final a JSON
import json
json_data = json.dumps(result, indent=4, ensure_ascii=False)

# Mostrar el JSON por consola (opcional)
print(json_data)




[
    {
        "id_categoria": 1,
        "titulo_categoria": "Personal Sanitario",
        "preguntas": [
            {
                "id_pregunta": 16,
                "texto_pregunta": "¿Qué necesitas como Personal Sanitario?",
                "opciones": [
                    {
                        "id_opcion": 1,
                        "texto_opcion": "Manejo clínico de pacientes con vih"
                    },
                    {
                        "id_opcion": 2,
                        "texto_opcion": "Protocolo PEP"
                    },
                    {
                        "id_opcion": 3,
                        "texto_opcion": "Tratamientos (PREP, TAR)"
                    },
                    {
                        "id_opcion": 4,
                        "texto_opcion": "Prevención de infecciones oportunistas"
                    },
                    {
                        "id_opcion": 5,
                        "texto_opcion": "Consejería 

C:\Users\2012f\AppData\Local\Temp\ipykernel_8440\3158678865.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_sociosanitarios, conn)
C:\Users\2012f\AppData\Local\Temp\ipykernel_8440\3158678865.py:32: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x[['id_opcion', 'texto_opcion']].to_dict(orient='records'))
C:\Users\2012f\AppData\Local\Temp\ipykernel_8440\3158678865.py:40: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future vers

## Sociosanitario


In [ ]:
# Escribe la consulta SQL
query_sociosanitarios = """
    SELECT 
        c.id_categoria,
        p.id_pregunta,
        o.id_opcion,
        c.titulo_categoria,
        p.texto_pregunta,
        o.texto_opcion
    FROM 
        categorias_chatbot c
    JOIN 
        categoria_pregunta_chat_intermed cp ON c.id_categoria = cp.id_categoria
    JOIN 
        preguntas_chatbot p ON cp.id_pregunta = p.id_pregunta
    JOIN 
        preguntas_opciones_chatbot po ON p.id_pregunta = po.id_pregunta
    JOIN 
        opciones_chatbot o ON po.id_opcion = o.id_opcion
    WHERE 
        c.seccion = 'sociosanitario'
    ORDER BY 
        c.id_categoria, p.id_pregunta, o.id_opcion;
"""


df = pd.read_sql_query(query_usuarios, conn)
df

# OTRO

In [ ]:
import json

# Agrupar opciones por pregunta y categoría
grouped = df.groupby(["id_categoria", "titulo_categoria", "texto_pregunta"])["texto_opcion"].apply(list).reset_index()

# Crear el JSON dinámico
json_data = {}
for _, row in grouped.iterrows():
    id_categoria = row["id_categoria"]
    titulo = row["titulo_categoria"]
    pregunta = row["texto_pregunta"]
    opciones = row["texto_opcion"]
    
    if id_categoria not in json_data:
        json_data[id_categoria] = {"titulo": titulo, "preguntas": {}}
    
    json_data[id_categoria]["preguntas"][pregunta] = opciones

# Convertir a JSON
json_resultado = json.dumps(json_data, indent=4, ensure_ascii=False)

print(json_resultado)

In [12]:
import json

# Agrupar opciones por pregunta y categoría
grouped = df.groupby(["id_categoria", "titulo_categoria", "texto_pregunta"])["texto_opcion"].apply(list).reset_index()

# Crear el JSON dinámico
json_data = {}
for _, row in grouped.iterrows():
    id_categoria = row["id_categoria"]
    titulo = row["titulo_categoria"]
    pregunta = row["texto_pregunta"]
    opciones = row["texto_opcion"]
    
    if id_categoria not in json_data:
        json_data[id_categoria] = {"titulo": titulo, "preguntas": {}}
    
    json_data[id_categoria]["preguntas"][pregunta] = opciones

# Convertir a JSON
json_resultado = json.dumps(json_data, indent=4, ensure_ascii=False)

print(json_resultado)


{
    "1.1": {
        "titulo": "Tengo VIH",
        "preguntas": {
            "¿Cuándo te diagnosticaron?": [
                "Hace menos de 6 meses",
                "Entre 6 meses y un año",
                "Hace más de un año"
            ],
            "¿Estás en tratamiento TAR?": [
                "Sí",
                "No",
                "No estoy seguro"
            ],
            "¿Tienes acceso a un médico?": [
                "Sí",
                "No"
            ]
        }
    },
    "1.2": {
        "titulo": "Creo que me he expuesto al virus",
        "preguntas": {
            "¿Cuándo ocurrió la posible infección?": [
                "Últimas 72h",
                "Hace más de 72h"
            ]
        }
    },
    "2.1": {
        "titulo": "Personal sanitario",
        "preguntas": {
            "¿que necesitas?": [
                "Manejo clínico de pacientes con VIH",
                "Protocolo PEP"
            ]
        }
    },
    "2.2": {
        "titulo

In [12]:
import pandas as pd

# Tabla de categorías
categorias = pd.DataFrame({
    "id_categoria": [1, 2, 3],
    "titulo_categoria": ["Tengo VIH", "Creo que me he expuesto al virus", "Personal sanitario"]
})

# Tabla de preguntas
preguntas = pd.DataFrame({
    "id_pregunta": [1, 2, 3, 4, 5],
    "id_categoria": [1, 1, 1, 2, 3],
    "texto_pregunta": [
        "¿Cuándo te diagnosticaron?",
        "¿Estás en tratamiento TAR?",
        "¿Tienes acceso a un médico?",
        "¿Cuándo ocurrió la posible infección?",
        "¿Qué necesitas?"
    ]
})

# Tabla de opciones
opciones = pd.DataFrame({
    "id_opcion": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    "id_pregunta": [1, 1, 1, 2, 2, 2, 3, 3, 4, 4, 5, 5],
    "texto_opcion": [
        "Hace menos de 6 meses",
        "Entre 6 meses y un año",
        "Hace más de un año",
        "Sí",
        "No",
        "No estoy seguro",
        "Sí",
        "No",
        "Últimas 72h",
        "Hace más de 72h",
        "Manejo clínico de pacientes con VIH",
        "Protocolo PEP"
    ]
})



In [13]:

# 1. Unir preguntas con categorías
preguntas_categorias = pd.merge(
    preguntas,
    categorias,
    on="id_categoria",
    how="inner"
)

# 2. Unir con opciones
final_table = pd.merge(
    opciones,
    preguntas_categorias,
    on="id_pregunta",
    how="inner"
)

In [16]:
# Lista de columnas que deben ir primero
columns_first = ["id_categoria", "id_pregunta", "id_opcion"]

# Añade las columnas restantes
columns_rest = [col for col in final_table.columns if col not in columns_first]

# Reordena las columnas
final_table = final_table[columns_first + columns_rest]

print(final_table)


    id_categoria  id_pregunta  id_opcion                         texto_opcion  \
0              1            1          1                Hace menos de 6 meses   
1              1            1          2               Entre 6 meses y un año   
2              1            1          3                   Hace más de un año   
3              1            2          4                                   Sí   
4              1            2          5                                   No   
5              1            2          6                      No estoy seguro   
6              1            3          7                                   Sí   
7              1            3          8                                   No   
8              2            4          9                          Últimas 72h   
9              2            4         10                      Hace más de 72h   
10             3            5         11  Manejo clínico de pacientes con VIH   
11             3            

In [17]:
final_table

,id_categoria,id_pregunta,id_opcion,texto_opcion,texto_pregunta,titulo_categoria
0,1,1,1,Hace menos de 6 meses,¿Cuándo te diagnosticaron?,Tengo VIH
1,1,1,2,Entre 6 meses y un año,¿Cuándo te diagnosticaron?,Tengo VIH
2,1,1,3,Hace más de un año,¿Cuándo te diagnosticaron?,Tengo VIH
3,1,2,4,Sí,¿Estás en tratamiento TAR?,Tengo VIH
4,1,2,5,No,¿Estás en tratamiento TAR?,Tengo VIH
5,1,2,6,No estoy seguro,¿Estás en tratamiento TAR?,Tengo VIH
6,1,3,7,Sí,¿Tienes acceso a un médico?,Tengo VIH
7,1,3,8,No,¿Tienes acceso a un médico?,Tengo VIH
8,2,4,9,Últimas 72h,¿Cuándo ocurrió la posible infección?,Creo que me he expuesto al virus
9,2,4,10,Hace más de 72h,¿Cuándo ocurrió la posible infección?,Creo que me he expuesto al virus


In [15]:
import json

# Agrupar opciones por pregunta y categoría
grouped = df.groupby(["id_categoria", "titulo_categoria", "texto_pregunta"])["texto_opcion"].apply(list).reset_index()

# Crear el JSON dinámico
json_data = {}
for _, row in grouped.iterrows():
    id_categoria = row["id_categoria"]
    titulo = row["titulo_categoria"]
    pregunta = row["texto_pregunta"]
    opciones = row["texto_opcion"]
    
    if id_categoria not in json_data:
        json_data[id_categoria] = {"titulo": titulo, "preguntas": {}}
    
    json_data[id_categoria]["preguntas"][pregunta] = opciones

# Convertir a JSON
json_resultado = json.dumps(json_data, indent=4, ensure_ascii=False)

print(json_resultado)


{
    "1.1.1": {
        "titulo": "Tengo VIH",
        "preguntas": {
            "¿Cuándo te diagnosticaron?": [
                "Hace menos de 6 meses"
            ]
        }
    },
    "1.1.2": {
        "titulo": "Tengo VIH",
        "preguntas": {
            "¿Cuándo te diagnosticaron?": [
                "Entre 6 meses y un año"
            ]
        }
    },
    "1.1.3": {
        "titulo": "Tengo VIH",
        "preguntas": {
            "¿Cuándo te diagnosticaron?": [
                "Hace más de un año"
            ]
        }
    },
    "1.1.4": {
        "titulo": "Tengo VIH",
        "preguntas": {
            "¿Estás en tratamiento TAR?": [
                "Sí"
            ]
        }
    },
    "1.1.5": {
        "titulo": "Tengo VIH",
        "preguntas": {
            "¿Estás en tratamiento TAR?": [
                "No"
            ]
        }
    },
    "1.1.6": {
        "titulo": "Tengo VIH",
        "preguntas": {
            "¿Estás en tratamiento TAR?": [
     

## probar base de datos a json

In [ ]:


except psycopg2.OperationalError as e:
    raise HTTPException(status_code=400, detail="Error de conexión: " + str(e))
except Exception as error:
    raise HTTPException(status_code=400, detail="Error desconocido: " + str(e))
    
# Escribe la consulta SQL
query = "SELECT * FROM sociosanitarios_data"

# Usa pandas para ejecutar la consulta y convertirla en un DataFrame
df = pd.read_sql_query(query, connection)

# Agrupar opciones por pregunta y categoría
grouped = df.groupby(["id_categoria", "titulo_categoria", "texto_pregunta"])["texto_opcion"].apply(list).reset_index()

# Crear el JSON dinámico
json_data = {}
for _, row in grouped.iterrows():
    id_categoria = row["id_categoria"]
    titulo = row["titulo_categoria"]
    pregunta = row["texto_pregunta"]
    opciones = row["texto_opcion"]
    
    if id_categoria not in json_data:
        json_data[id_categoria] = {"titulo": titulo, "preguntas": {}}
    
    json_data[id_categoria]["preguntas"][pregunta] = opciones

# Convertir a JSON
json_resultado = json.dumps(json_data, indent=4, ensure_ascii=False)

print(json_resultado)
